# Problem

Set covering problem is a classical optimization problem. It consists of finding the smallest set of subsets of a given set S such that each element of S belongs to at least one of these subsets.

# Implementation

In [457]:
import random
import sys
import numpy as np

sys.path.append("../../")
from lib.comparison import comparison
from lib.search import Search

## Standard Implementation

### Class Definition

In [458]:
class SetCovering(Search):
    """
    state: set of indices
    """
    def is_goal(self, state):
        total = 0
        length = len(self.space[0])
        for i in range(length):
            for j in state:
                if self.space[j][i]:
                    total += 1
                    break
        if total == length:
            return True
        else:
            return False

    def actions(self, state):
        """Return the actions that can be executed in the given state."""
        actions = []
        for i in range(len(self.space)):
            if i not in state:
                actions.append(i)
        return actions

    """ def solution_format(self, actions):
        result = []
        for i in actions:
            result.append(self.space[i])
        return result """

    def results(self, state, action):
        """Return the state that results from executing the given
        action in the given state."""
        new_state = state.copy()
        new_state.add(action)
        return new_state

### Informed Strategies Functions

In [459]:
def h_remaining(self, node):
    total = 0
    length = len(self.space[0])
    for i in range(length):
        for j in node.state:
            if self.space[j][i]:
                total += 1
                break
    return length - total

In [460]:
def h_least_remaining(self, node):
    total = 0
    length = len(self.space[0])
    for i in range(length):
        for j in node.state:
            if self.space[j][i]:
                total += 1
                break
    return total

In [461]:
def h_unique(self,node,weight=3):
    s=list(node.state)
    sp=np.array(self.space)
    total=sp.sum(axis=0)
    r=np.any(sp[s, :], axis=0)
    weighted=(r*(1/total))
    return -weighted.sum()*weight

## Numpy Implementation

### Class Definition

In [462]:
class NumpySetCovering(Search):
    """
    state: set of indices
    """

    def is_goal(self, state):
        total = 0
        length = self.space.shape[1]
        total = np.any(self.space[list(state), :], axis=0).sum()
        if total == length:
            return True
        else:
            return False

    def actions(self, state):
        """Return the actions that can be executed in the given state."""
        actions = []
        for i in range(self.space.shape[0]):
            if i not in state:
                actions.append(i)
        return actions

    """ def solution_format(self, actions):
        result = []
        for i in actions:
            result.append(self.space[i])
        return result """

    def results(self, state, action):
        """Return the state that results from executing the given
        action in the given state."""
        new_state = state.copy()
        new_state.add(action)
        return new_state

### Informed Strategies Functions

In [463]:
def numpy_h_remaining(self, node):
    total = 0
    length = self.space.shape[1]
    total = np.any(self.space[node.state, :], axis=0).sum()
    return length - total

In [464]:
def numpy_h_least_remaining(self,node):
    total = np.any(self.space[node.state, :], axis=0).sum()
    return total

In [465]:
def numpy_unique(self,node,weight=1):
    total=self.space.sum(axis=0)
    r=np.any(self.space[node.state, :], axis=0)
    weighted=(r*(1/total))
    return -weighted.sum()

## State Space Generation

In [466]:
def generate_random_space(set_number, problem_shape, probability, return_numpy=False):
    matrix=[
            [random.random() < probability for i in range(problem_shape)]
            for j in range(set_number)
        ]
    return matrix, np.array(matrix)

# Tests

## Test 1

In [467]:
SUBSETS_NUMBER = 10
SUBSETS_SIZE = 5
PROBABILITY = 0.3

In [468]:
space, nspace = generate_random_space(SUBSETS_NUMBER, SUBSETS_SIZE, PROBABILITY)
print(nspace)

[[False False False False  True]
 [False  True False False False]
 [False False False False False]
 [False False False  True False]
 [False False  True False False]
 [False False False False False]
 [False  True False  True False]
 [ True  True  True False  True]
 [False False  True False False]
 [ True False False False False]]


### Standard Implementation

In [469]:
m = SetCovering(space)

In [470]:
initial_space = set()

In [471]:
comparison(m, initial_space, ["bfs", "dfs", "greedy", "astar"], heuristics=[(h_remaining, "Most Remaining"), (h_least_remaining, "Least Remaining"), (h_unique, "Unique")])

+-----------------------------+--------------+----------+-----------+
|           Strategy          |   Solution   | Explored |    Time   |
+-----------------------------+--------------+----------+-----------+
|             bfs             |    [3, 7]    |    39    | 1.885e-03 |
|             dfs             | [9, 8, 7, 6] |    5     | 9.608e-05 |
|  greedy | h: Most Remaining |    [7, 3]    |    3     | 4.280e-04 |
| greedy | h: Least Remaining | [2, 5, 3, 7] |   610    | 1.588e-01 |
|      greedy | h: Unique     |    [7, 3]    |    3     | 1.184e-03 |
|  astar | h: Most Remaining  |    [7, 3]    |    3     | 9.799e-05 |
|  astar | h: Least Remaining |    [3, 7]    |   198    | 3.146e-02 |
|      astar | h: Unique      |    [7, 3]    |    3     | 6.070e-04 |
+-----------------------------+--------------+----------+-----------+


### NumPy Implementation

In [472]:
n = NumpySetCovering(nspace)

In [473]:
initial_space = set()

In [474]:
comparison(n, initial_space, ["bfs", "dfs", "greedy", "astar"], heuristics=[(numpy_h_remaining, "Most Remaining"),(numpy_unique, "Unique")])

+----------------------------+--------------+----------+-----------+
|          Strategy          |   Solution   | Explored |    Time   |
+----------------------------+--------------+----------+-----------+
|            bfs             |    [3, 7]    |    45    | 3.964e-03 |
|            dfs             | [9, 8, 7, 6] |    5     | 1.450e-04 |
| greedy | h: Most Remaining |    [7, 3]    |    3     | 2.649e-04 |
|     greedy | h: Unique     |    [7, 3]    |    3     | 9.162e-04 |
| astar | h: Most Remaining  |    [7, 3]    |    3     | 2.329e-04 |
|     astar | h: Unique      |    [7, 3]    |    3     | 6.070e-04 |
+----------------------------+--------------+----------+-----------+


# Analysis

## Search Strategies

- The standard implementation of the problem is more efficient than the NumPy implementation.
- The informed strategies are more efficient than the uninformed ones.
- DFS is significantly more efficient than BFS, altough it doesn't find the optimal solution in most cases.
- greedy search is the most efficient informed strategy, altough it doesn't always find the optimal solution.
- A* efficiency is similar to greedy search, but it finds the optimal solution in most cases.

## Heuristics

- `h_remaining`: returns the number of elements that are not covered by the current state.
- `h_least_remaining`: returns the number of elements that are covered by the current state.
- `h_unique`: returns the sum of each element that is covered in the current state inversely multiplied by the frequency of that element on the state space.
  - Example:
    - s1=[1, 0, 1, 0]
    - s2=[0, 1, 0, 1]
    - space=[[1, 0, 1, 0],
            [0, 1, 0, 1],
            [0, 1, 0, 0]]
    - h_unique(s1) = -(1*1+0*1/2+1*1+0*1) = -2
    - h_unique(s2) = -(0*1+1*1/2+0*1+1*1) = -1.5
    - Therefore, s1 is chosen over s2.

- h_least_remaining obtained significantly worse results than the other functions, which was expected as it always selects the state with the least number of remaining elements to be covered.
- h_remaining and h_unique obtained very similar results. Sometimes one of them outperform the other, but overall they are very similar.